In [2]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [3]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from src.dataset.dataset import RawData
from src.dataset.wikisection_preprocessing import (
    tokenize,
    clean_sentence,
    preprocess_text_segmentation,
    format_data_for_db_insertion,
)
from src.dataset.utils import truncate_by_token
from db.dbv2 import Table, AugmentedTable, TrainTestTable
import pprint

from src.bertkeywords.src.similarities import Embedding, Similarities
from src.bertkeywords.src.keywords import Keywords
from src.encoders.coherence import Coherence

<IPython.core.display.Javascript object>

In [33]:
coherence = Coherence(max_words_per_step=4)

2023-03-27 22:17:47.536130: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-27 22:17:48.326214: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the chec

<IPython.core.display.Javascript object>

In [9]:
sent1 = "this is a sentence"
sent2 = "this is a similar sentence"
sent3 = "another sentence with some structure"
sent4 = "the structure is not that sound"
sent5 = "especially when it comes to architectural sound"

segment1 = [sent1, sent2, sent3, sent4, sent5]
segment2 = [sent1, sent2, sent5, sent4, sent3]
segment3 = [sent2, sent3, sent1, sent4, sent3]

<IPython.core.display.Javascript object>

In [10]:
coherence.get_coherence(segment1)

['sentence', 'this', 'sentence', 'structure', 'sound']

<IPython.core.display.Javascript object>

In [11]:
coherence.get_coherence_map([segment1, segment2, segment3])

[['sentence', 'this', 'sentence', 'structure', 'sound'],
 ['sentence', 'this', 'sound', 'structure'],
 ['sentence', 'sentence', 'is', 'structure']]

<IPython.core.display.Javascript object>

## Test with Data

In [12]:
dataset_type = "city"
table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

<IPython.core.display.Javascript object>

In [13]:
data = table.get_all()

text_data = [x[1] for x in data]
text_labels = [x[2] for x in data]

<IPython.core.display.Javascript object>

In [27]:
all_segments = table.get_all_segments()
text_segments = [[y[1] for y in x] for x in all_segments]
segments_labels = [[1 if i == 0 else 0 for i, y in enumerate(x)] for x in all_segments] # [[1,0,0], [1,0], [1,0,0,0], ...]

<IPython.core.display.Javascript object>

In [34]:
segments_to_test = 5
max_tokens = 400  # want to keep this under 512

for segment, labels in zip(
    text_segments[:segments_to_test], segments_labels[:segments_to_test]
):
    truncated_segment = [truncate_by_token(s, max_tokens) for s in segment]
    print(coherence.get_coherence(truncated_segment))

['navarre', 'district', 'railway', 'buildings', 'festival', 'festival', 'parade', 'festival', 'basque', 'event', 'event', 'festival', 'sebastián']
['census', 'households', 'census', 'household', 'population']
[]
['census', 'households', 'census', 'household', 'population']
[]


<IPython.core.display.Javascript object>